In [62]:
import psycopg2

pg_host = "localhost"
pg_database = "demo"
pg_user = "postgres"
pg_password = "postgres"

# Establish PostgreSQL connection
pg_conn = psycopg2.connect(
    host=pg_host,
    database=pg_database,
    user=pg_user,
    password=pg_password
)
pg_cursor = pg_conn.cursor()

In [63]:
pg_cursor.execute("""
select d_source.id, d_target.id, (2*count(*)) as count from sentence_matching
inner join datasets as ds_source on ds_source.id = sentence_matching.source_id
inner join datasets as ds_target on ds_target.id  = sentence_matching.target_id 
inner join "document" as d_source on d_source.name = ds_source.app 
inner join "document" as d_target on d_target.name = ds_target.app
where "method" = 'COSINE' and embedding = 'text-embedding-ada-002' and ds_source.app != ds_target.app
group by d_source."id", d_target."id"
""")

combination = pg_cursor.fetchall()

In [64]:
for index, row in enumerate(combination):
  pg_cursor.execute("UPDATE document_redundant SET n_intersection = %s WHERE source_id = %s and target_id = %s", (row[2], row[0], row[1]))
  pg_conn.commit()
  